In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from collections import namedtuple

import os
import pickle

In [2]:
# path to the chromedriver executable
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

top_songs_genius = "https://genius.com/#Top-Songs"
driver = webdriver.Chrome(chromedriver)

driver.get(top_songs_genius)

In [ ]:
# Use Selenium to pull many links from the Top Songs page
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")




In [ ]:
# This code snippet is there to grab links to songs
# on the https://genius.com/#TopSongs page.

link_selector = '//a[@class= "chart_row"]'
top_songs = driver.find_elements_by_xpath(link_selector)



link_list = []

for elem in top_songs:
    link_list.append(elem.get_attribute('href'))

In [ ]:
link_list = [tag.get('href') for tag in soup.find_all('a')]

just_links = link_list[53:300:3]

main_link = 'https://www.boxofficemojo.com{}'
full_links = [main_link.format(link) for link in just_links]

In [43]:
# Scroll to the bottom of the screen so that the metadata appears
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# May need to plug in some kind of pause.
# Sometimes the metadata doesn't appear right away.

all_metadata = driver.find_elements_by_class_name('metadata_with_icon')

# This is just to check whether the metadata is there.

for item in all_metadata:
    print(item.text)

# This creates a list of the main metadata categories in string form
# This includes the number of views, the number of contributors
# and the tags (= genres)

metadata_list = []

for item in all_metadata:
    metadata_list.append(item.text)

In [ ]:
# Pickle the list of links scraped above:
with open('links.pkl', 'wb') as pickle_file:
    pickle.dump(link_list, pickle_file)

In [3]:
# Unpickle the list of links
with open('links.pkl', 'rb') as pickle_file:
    link_list = pickle.load(pickle_file)

In [47]:
# Define the Song data structure
# Remember: namedtuples require that you plug in every data type.
Song = namedtuple('Song', ['title', 'total_words', 'unique_words', 'words_per_minute', 'contributors', 'views', 'tags','lyrics'])

In [48]:

for link_anchor in driver.find_elements_by_xpath(link_selector):
    print(link_anchor.text)

In [10]:
url = 'https://genius.com/Eminem-rap-god-lyrics'

response = requests.get(url)
tag_soup = bs4.BeautifulSoup(response.text)

lyrics = ''
for lyric in tag_soup.find_all(class_='song_body-lyrics'):
    lyrics += lyric.text

# Removes the bracketed sections like [Intro: Ariana Grande]
# Still need to work on removing header and footer
# header = title, i.e. MONOPOLY by Ariana Grande
# Footer = advertisement at the end: 'More on Genius'

cleaned_lyrics = re.sub(r'\[.+\]', '', lyrics)
cleaned_lyrics = re.sub(r'[\'\"\(\),\?\“\”]', '', cleaned_lyrics.lower())

# Get a list of the words in the lyrics
words_list = cleaned_lyrics.lower().split()

# Freq dict
freq_dict = {}
for word in words_list:
    freq_dict[word] = freq_dict.get(word, 0) + 1

In [115]:
genius_url = "https://genius.com/Eminem-rap-god-lyrics"
driver = webdriver.Chrome(chromedriver)

driver.get(genius_url)

In [103]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [28]:
all_metadata = driver.find_elements_by_class_name('metadata_with_icon')

metadata_list = []

for item in all_metadata:
    metadata_list.append(item.text)

In [53]:
views1 = metadata_list[1]
contrib = metadata_list[2]
tags1 = metadata_list[3]

In [58]:
# Still need to figure out how to scrape duration of song
# so that I can produce words_per_minute
rap_god = Song(title='Rap God', total_words=len(words_list), unique_words=len(
    freq_dict), lyrics=cleaned_lyrics, views=views1, contributors=contrib, tags=tags1, words_per_minute=123)

In [65]:
print(rap_god.lyrics)


rap god lyrics




look i was gonna go easy on you not to hurt your feelings.
but im only going to get this one chance.
somethings wrong i can feel it.
six minutes. six minutes. six minutes slim shady youre on!
just a feeling ive got. like somethings about to happen but i dont know what. if that means what i think it means were in trouble big trouble; and if he is as bananas as you say im not taking any chances.
you are just what the doc ordered.


im beginnin to feel like a rap god rap god
all my people from the front to the back nod back nod
now who thinks their arms are long enough to slap box slap box
they said i rap like a robot so call me rap-bot


but for me to rap like a computer it must be in my genes
i got a laptop in my back pocket
my penll go off when i half-cock it
got a fat knot from that rap profit
made a livin and a killin off it
ever since bill clinton was still in office
with monica lewinsky feelin on his nutsack
im an mc still as honest
but as rude and as indecent a

In [105]:
xpath = '/html/body/apple-music-player/div/div/div[2]/playback-time[2]/div'

In [106]:
# This didn't get me anywhere.
# But the duration should live in this little player!
music_player = driver.find_elements_by_xpath(xpath)

for elem in music_player:
    print(elem.text)

In [107]:
music_player

[]

In [118]:
driver.switch_to.frame('apple_music_player')

AttributeError: 'WebDriver' object has no attribute 'switchto'

In [108]:
html = driver.page_source
soup = BeautifulSoup(html)

In [109]:
soup.find_all('apple-music-player')

[]

In [ ]:
/html/body/routable-page/ng-outlet/song-page/div/apple-music-player/div/div/iframe